In [1]:
# -*- coding:utf-8 -*-

In [2]:
import os
import sys
import cv2
import numpy as np
from skimage import filters,io
import matplotlib.pyplot as plt

In [3]:
filename = "Char_Index.txt"
INPUT_DIR = "Char_Image"
OUTPUT_DIR = "Char_Image_Binary"

##  PreProcessing

In [82]:
input1 = []
input2 = []
input3 = []
#Efield = []
with open(filename, 'r') as  file_to_read:
    while True:
        # 整行读取数据
        lines = file_to_read.readline()
        if not lines:
            break
            pass
        # 将整行数据分割处理，这里分隔符为空格
        tmp1, tmp2, tmp3 = [i for i in lines.split()]
        input1.append(tmp1)
        input2.append(tmp2)
        input3.append(tmp3)
    input1 = np.array(input1[1:])
    input2 = np.array(input2[1:])
    input3 = np.array(input3[1:])
    indexFileName = input3

for k in range(0, len(indexFileName)):
    path = os.path.join(INPUT_DIR, indexFileName[k])
    image = io.imread(path, as_gray=True)
    #image = cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2GRAY)
    thresh = filters.threshold_otsu(image)
    # 乘以1.0是因为将True,False转换为浮点数
    dst = (image > thresh) * 1
    a, b = dst.shape
    # # 绘图显示
    # plt.figure("thresh", figsize=(8,8))
    #
    # plt.subplot(121)
    # plt.title("Original")
    # plt.imshow(image, plt.cm.gray)
    #
    # plt.subplot(122)
    # plt.title("binary")
    # # 0表示黑,1表示白
    # plt.imshow(dst, plt.cm.gray)
    # plt.show()

    # 四个角的二值化数据大于2,白底
    if (dst[0,0] + dst[0, b-1] + dst[a-1, 0] + dst[a-1,b-1] >= 2):
        for i in range(0,a):
            for j in range(0,b):
                # 将每一个像素的值反向,转换成黑底白字
                dst[i,j] = 1 - dst[i,j]
    path_out = os.path.join(OUTPUT_DIR, str(k+1)+".jpg")
    dst = dst.astype(np.uint8) * 255
    io.imsave(path_out, dst)
    #io.imshow(dst)

## PreProcessingToCorrect 

In [84]:
input1_ = []
input2_ = []
with open("Char_Index_Err.txt") as file_idx_err:
    while True:
        # 整行读取数据
        lines = file_idx_err.readline()
        if not lines:
            break
        # 将整行数据分割处理，这里分隔符为空格
        tmp1, tmp2 = [i for i in lines.split()]
        input1_.append(tmp1)
        input2_.append(tmp2)

    input1_ = np.array(input1_[1:])
    input2_ = np.array(input2_[1:])
    IndexFileName_Err = input2_

for k in range(0, len(IndexFileName_Err)):
    path_e = os.path.join(OUTPUT_DIR, IndexFileName_Err[k])
    img_e = io.imread(path_e)
    a, b = img_e.shape
    for i in range(0,a):
        for j in range(0,b):
            dst[i,j] = 255 - img_e[i,j]       
    io.imsave(path_e ,dst)
    #io.imshow(img_e)  

## Feature Extration

### Feature1

找出每一行\每一列的白色点的个数

In [30]:
with open("feature1.txt",'w+') as fd:
    for k in range(0, 1000):
        path = os.path.join(OUTPUT_DIR, str(k+1)+'.jpg')
        image = io.imread(path)
        thresh = filters.threshold_otsu(image)
        dst = (image > thresh) * 1
        a, b = dst.shape
        C = np.zeros(a+b, dtype=np.int)

        # 每一行
        for i in range(0, a):
            for j in range(0, b):
                if(dst[i,j] == 1):
                    C[i] += 1
        # 每一列
        for j in range(0, b):
            for i in range(0, a):
                if(dst[i,j] == 1):
                    C[a+j] += 1

        fd.write(str(k+1) + '\n')
        for i in range(0, a+b-1):
            fd.write(str(C[i]) + ',')
        if k < 999:
            fd.write(str(C[a+b-1]) + '\n')
        else:
            fd.write(str(C[a+b-1]))
